# Dealing with Images (Enhancing and Segmenting) [Notebook 5]

## Introduction

This project dives into Encoders-Decoders, where these models are used to edit and generate full images. How these models can be adapted for a wider range of applications such as image denoising or object and instance segmentation. The project will also introduce new concepts like Unpooling, Transposed and Atrous Convolutions layers to the network architecture and its utility for high-dimensional data. Encoders-Decoders can be used for semantic segmentation for driverless cars, where it would help in defining the objects surrounding the vehicle like, roads, other vehicles, people or trees etc. 

## Breakdown of this Project:
- Introduction to Encoders-Decoders. (Notebook 1)
- Encoders-Decoders trained for pixel-level prediction. (Notebook 1)
- Layers such as Unpooling, Transposed and Atrous Convolutions to output high-dimensional data. (Notebook 2)
- FCN (Notebook 3) and U-Net (Notebook 4) Architectures for semantic segmentation. 
- Instance segmentation (extension of Faster-RCNN with Mask-RCNN) (Notebook 5)

## Requirements:
1) Tensorflow 2.0 (GPU prefferably) \
2) CV2 (OpenCV) \
3) Cython \
4) Eigen \
5) PyDenseCRF

For "PyDenseCRF" for windows, LINK: https://github.com/lucasb-eyer/pydensecrf\

It can be installed directly with the following in command prompt or terminal-equivalent: __conda install -c conda-forge pydensecrf__.

If Conda-Forge __does not work__, try: 
- going to: https://www.lfd.uci.edu/~gohlke/pythonlibs/#pydensecrf
- Download: pydensecrf-1.0rc2-cp37-cp37m-win_amd64.whl
- Where "cp37" in the filename is the python version of 3.7, make sure you download the correct one.
- Place the downloaded "pydensecrf-1.0rc2-cp37-cp37m-win_amd64.whl" file in your working directory drive.
- Open Command Prompt and type in: pip install pydensecrf-1.0rc2-cp37-cp37m-win_amd64.whl
- Or if you placed it in a folder or different location: pip install <FILEPATH>\pydensecrf-1.0rc2-cp37-cp37m-win_amd64.whl

## Dataset:
    



### Import the required libraries:

In [1]:
%matplotlib inline

import tensorflow as tf
import numpy as np
import math
import timeit
import time
import os
import matplotlib.pyplot as plt

# Run on GPU:
os.environ["CUDA_VISIBLE_DEVICES"]= "0" 


In [2]:
# Set the random set seed number: for reproducibility.
Seed_nb = 42

# Set to run or not run the code block: for code examples only. (0 = run code, and 1 = dont run code)
dont_run = 0

### GPU Information:

In [3]:
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))
devices = sess.list_devices()
devices

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device



[_DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 268435456, 8390128233705279065),
 _DeviceAttributes(/job:localhost/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 4832671986718658171),
 _DeviceAttributes(/job:localhost/replica:0/task:0/device:XLA_GPU:0, XLA_GPU, 17179869184, -5649852988599655110)]

### Use RTX_GPU Tensor Cores for faster compute: FOR TENSORFLOW ONLY

Automatic Mixed Precision Training in TF. Requires NVIDIA DOCKER of TensorFlow.

Sources:
- https://developer.nvidia.com/automatic-mixed-precision
- https://docs.nvidia.com/deeplearning/performance/mixed-precision-training/index.html#framework

When enabled, automatic mixed precision will do two things:

- Insert the appropriate cast operations into your TensorFlow graph to use float16 execution and storage where appropriate(this enables the use of Tensor Cores along with memory storage and bandwidth savings). 
- Turn on automatic loss scaling inside the training Optimizer object.

In [4]:
# os.environ['TF_ENABLE_AUTO_MIXED_PRECISION'] = '1'

EXAMPLE CODE: 

In [5]:
# # Graph-based example:
# opt = tf.train.AdamOptimizer()
# opt = tf.train.experimental.enable_mixed_precision_graph_rewrite(opt)
# train_op = opt.miminize(loss)

# # Keras-based example:
# opt = tf.keras.optimizers.Adam()
# opt = tf.train.experimental.enable_mixed_precision_graph_rewrite(opt)
# model.compile(loss=loss, optimizer=opt)
# model.fit(...)

### Use RTX_GPU Tensor Cores for faster compute: FOR KERAS API

Source:
- https://www.tensorflow.org/guide/keras/mixed_precision
- https://www.tensorflow.org/api_docs/python/tf/keras/mixed_precision/experimental/Policy

In [6]:
# from tensorflow.keras.mixed_precision import experimental as mixed_precision

In [7]:
# # Set for MIXED PRECISION:
# policy = mixed_precision.Policy('mixed_float16')
# mixed_precision.set_policy(policy)

# print('Compute dtype: %s' % policy.compute_dtype)
# print('Variable dtype: %s' % policy.variable_dtype)

### To run this notebook without errors, the GPU will have to be set accordingly:

In [8]:
# physical_devices = tf.config.list_physical_devices('GPU') 
# physical_devices
# tf.config.experimental.set_memory_growth(physical_devices[0], True) 

## 1 - Instance segmentation with Mask-RCNN (extension of Faster-RCNN):

Mask-RCNN is an extension of Faster-RCNN, where instance segmentation can be achieved in two steps:
1. Utilising a Object Detection model, like Faster-RCNN, to output the bounding boxes for each of the instance of the target classes.
2. These instances are then passed to a semantic segmentation model to obtain the instance masks.

This is an advantageous approach to solving the instance segmentation task because it assumes that the predicted bounding boxes are accurate, where the element within the bounding box will be classified pixel-wise to the belonging class. 

Mask-RCNN's architecture can be describe as a whole pipeline with two networks being stitched together, and is trained in an end-to-end manner. The segmentation loss is backpropagated though the common layers of the two network to ensure that the features that are extracted are useful for both the detection and segmentation tasks.

##### Below shows the Mask-RCNN Architecture:

<img src="Description Images/Mask_RCNN.PNG" width="450">

Image Ref -> https://www.researchgate.net/figure/The-structure-of-the-Mask-R-CNN-architecture_fig2_337795870

As mentioned, Mask-RCNN is based on the Faster-RCNN model, where it is composed of a Region Proposal Network (RPN), that is followed by two branches that predicts the class the box offset for each of the proposed region. To achieve segmentation with masks, this model has a third branch that outputs the binary mask for the element in each region. It should be noted that this branch is composed of a couple standard and transposed convolutions. Overall, this model achieves instance segmentation by processing everything in parallel rather than sequential. 




## 2 - Implementing Mask-RCNN with Tensorflow Object Detection API:

### 2.1 - TensorFlow Object Detection API:

Using the API allows the access to the most up to date implementation and the model would be pre-trained. 

The following code follows the step-by-step guide from the link: https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/install.html

### 2.2 - Utilising a Pretrained Model:

The API consists of several pre-trained models with the COCO dataset. The model themselves can vary in base architecture, such as different parameters or backbones. Depending on the choice model, it can vary in inference speed and performance. Note, as a rule of thumb, the inference time grows with the increase in mean average precision.

### 2.3 - Installation Process:

The step-by-step guide to set up the API from the link: https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/install.html


## 2.4 - Implementation:

In [9]:
# Import the required Libraries:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import pathlib
import time

from collections import defaultdict
from io import StringIO
from PIL import Image
from IPython.display import display

%matplotlib inline
import matplotlib.pyplot as plt

#### Import the Object Detection Modules from the API:

In [10]:
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

## 2.5 - Downloading the Testing Images:

Download the images and save them into the folder "data/image". The images will be from: https://github.com/tensorflow/models/tree/master/research/object_detection/test_images

In [11]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.chdir('D:\TensorFlow ObjDetection API')
tf.get_logger().setLevel('ERROR')  

In [12]:
# Enable GPU dynamic memory allocation
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [13]:
def download_images():
    base_url = 'https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/test_images/'
    filenames = ['image1.jpg', 'image2.jpg']
    image_paths = []
    for filename in filenames:
        image_path = tf.keras.utils.get_file(fname=filename,
                                            origin=base_url + filename,
                                            untar=False)
        image_path = pathlib.Path(image_path)
        image_paths.append(str(image_path))
    return image_paths


In [14]:
# Uncomment, if downloading the label maps are required:
# IMAGE_PATHS = download_images()
# TEST_IMAGE_PATHS = IMAGE_PATHS
# TEST_IMAGE_PATHS

In [15]:
# PATH_TO_TEST_IMAGES_DIR = os.path.abspath('models\research\object_detection\test_images')
PATH_TO_TEST_IMAGES_DIR = pathlib.Path('models/research/object_detection/test_images')
TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*.jpg")))
TEST_IMAGE_PATHS

[WindowsPath('models/research/object_detection/test_images/image1.jpg'),
 WindowsPath('models/research/object_detection/test_images/image2.jpg')]

## 2.6 - Prepare the Model:

#### Model Loader:

List of models can be found here: https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md

##### MODEL SELECTED -> Mask R-CNN Inception ResNet V2 1024x1024

To load in the selected model:

1. From the link above, select one of the model without clicking, as this will just download it. What is needed can be seen from step 2 and onwards.
2. Right click on the Model name of the model you would like to use.
3. Click on "Copy link address" to copy the download link of the model.
4. Paste the link in a text editor of your choice. You should observe a link similar to download.tensorflow.org/models/object_detection/tf2/YYYYYYYY/XXXXXXXXX.tar.gz.
5. Copy the XXXXXXXXX part of the link and use it to replace the value of the MODEL_NAME variable in the code shown below.
6. Copy the YYYYYYYY part of the link and use it to replace the value of the MODEL_DATE variable in the code shown below.

In [16]:
def load_model(model_name, model_date):
    """ This function loads and instantiates the requested model.
    Parameters:
        - model_name, is the (Str) of the model's name.
        - model_date, is the model date.
    Returns:
        - returns an instantiated model.
    Info:
        The models can be found here: https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md
    """
    # Download:
    base_url = 'http://download.tensorflow.org/models/object_detection/tf2/'
    model_file = model_name + '.tar.gz'
    model_dir = tf.keras.utils.get_file(fname=model_name,
                                        origin=base_url + model_date + '/' + model_file,
                                        untar=True)
    # Set the Path:
    PATH_TO_MODEL_DIR = str(model_dir)
    PATH_TO_SAVED_MODEL = PATH_TO_MODEL_DIR + "/saved_model"
        
    # Instantiate the Model:
    print('Loading model...', end='')
    start_time = time.time()
    
    model = tf.saved_model.load(PATH_TO_SAVED_MODEL)
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    print('Done! Took {} seconds'.format(elapsed_time))
    
#     model = model.signatures['serving_default']
        
    return model

In [17]:
# http://download.tensorflow.org/models/object_detection/tf2/20200711/mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8.tar.gz

In [18]:
detection_model = load_model(model_name = 'mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8', 
                             model_date= '20200711')

Loading model...Done! Took 86.36892080307007 seconds


In [19]:
# detection_model = load_model(model_name = 'faster_rcnn_resnet101_v1_1024x1024_coco17_tpu-8', 
#                              model_date= '20200711')

#### Load in the Label Map:

Downloads the labels file (.pbtxt) 

In [20]:
def download_labels(filename= 'mscoco_label_map.pbtxt'):
    """ This function downloads the labels from the dataset.
    Parameters:
        - filename, is the (Str) of the labels file (.pbtxt).
    Returns:
        - returns an instantiated model.
    Info:
        Full list of the labels files: https://github.com/tensorflow/models/tree/master/research/object_detection/data
    """
    
    base_url = 'https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/data/'
    label_dir = tf.keras.utils.get_file(fname=filename,
                                        origin=base_url + filename,
                                        untar=False)
    label_dir = pathlib.Path(label_dir)
    return str(label_dir)


In [21]:
# Uncomment, if downloading the label maps are required:
# LABEL_FILENAME = 'mscoco_label_map.pbtxt'
# PATH_TO_LABELS = download_labels(filename=LABEL_FILENAME)

In [22]:
PATH_TO_LABELS = 'models/research/object_detection/data/mscoco_label_map.pbtxt'

#### Load in the Label map data for visualisation:

The label maps will correspond with the index numbers to category names, therefore when the network perdicts a "5", it will mean an "airplane". The function used below is an internal utility function, that takes in anything that will returns a dictionary mapping of the integers to the respective string labels.

In [23]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS,
                                                                    use_display_name=True)

## 2.7 - Combining all the components together:

In [ ]:
# <img src="Description Images/.png" width="750">

# Image Ref -> 

# <img src="Description Images/.png" width="750">

# Image Ref -> 